<a href="https://colab.research.google.com/github/junghodavidlee/Project-3/blob/main/IMDB_Movie_Sentiment_Analysis_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install fastparquet
!pip install gradio
!pip install vaderSentiment
!pip install nltk
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from datasets import load_dataset
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
import gradio as gr
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import joblib
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.svm import LinearSVC
import spacy

In [4]:
# Importing the dataset
data = pd.read_csv("data/IMDB_Dataset.csv")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/IMDB_Dataset.csv'

In [5]:
# Import the IMDB dataset
#from google.colab import files
#uploaded = files.upload()


Saving IMDB Dataset.csv to IMDB Dataset.csv


In [7]:
# Import the IMDB dataset
data = pd.read_csv('data/IMDB_Dataset.csv')
data.head()

review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive

In [8]:
# Label encoding
data['sentiment'] = data['sentiment'].map({'positive': 1, 'negative': 0})
data['sentiment'].head()

0    1
1    1
2    1
3    0
4    1
Name: sentiment, dtype: int64

In [9]:
# Split the data into target and features
X = data['review']
y = data['sentiment']
# Do a train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

R2D2 Model

In [10]:
# Tokenizer parameters
MAX_VOCAB_SIZE = 10000  # Max words in vocabulary
MAX_SEQUENCE_LENGTH = 200  # Max length of each review
EMBEDDING_DIM = 100  # Word embedding size

# Tokenization
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post")

In [11]:
# LSTM model
model = Sequential([
    Embedding(input_dim=MAX_VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH),
    LSTM(128, return_sequences=True),  # LSTM layer
    Dropout(0.3),  # Prevent overfitting
    LSTM(64),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
# Fit the model
history = model.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_test_pad, y_test))

Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 577s 458ms/step - accuracy: 0.5155 - loss: 0.6902 - val_accuracy: 0.7379 - val_loss: 0.5606
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 729s 543ms/step - accuracy: 0.5989 - loss: 0.6378 - val_accuracy: 0.8021 - val_loss: 0.4470
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 621s 495ms/step - accuracy: 0.8091 - loss: 0.4301 - val_accuracy: 0.8146 - val_loss: 0.4615
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 581s 462ms/step - accuracy: 0.8625 - loss: 0.3506 - val_accuracy: 0.8694 - val_loss: 0.3144
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 615s 457ms/step - accuracy: 0.8997 - loss: 0.2613 - val_accuracy: 0.8701 - val_loss: 0.2966


In [13]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 43s 138ms/step - accuracy: 0.8702 - loss: 0.2950
Test Accuracy: 0.8701


In [14]:
# Example with sample review
sample_review = ["This movie was absolutely amazing!"]
sample_seq = tokenizer.texts_to_sequences(sample_review)
sample_pad = pad_sequences(sample_seq, maxlen=MAX_SEQUENCE_LENGTH, padding="post")
prediction = model.predict(sample_pad)

print("Positive" if prediction[0][0] > 0.5 else "Negative")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step
Positive


In [15]:
# Define the predict_review function
def predict_review(review):
    sample_seq = tokenizer.texts_to_sequences(review)
    sample_pad = pad_sequences(sample_seq, maxlen=MAX_SEQUENCE_LENGTH, padding="post")
    prediction = model.predict(sample_pad)
    return "Positive" if prediction[0][0] > 0.5 else "Negative"

In [16]:
# Run metrics on the model
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype(int)
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nAccuracy:", accuracy_score(y_test, y_pred))

313/313 ━━━━━━━━━━━━━━━━━━━━ 42s 134ms/step
Confusion Matrix:
 [[4465  496]
 [ 803 4236]]

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.90      0.87      4961
           1       0.90      0.84      0.87      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000


Accuracy: 0.8701


In [17]:
# Improved Gradio Interface
with gr.Blocks(css=".gradio-container {background-color: #266fd9; font-family: Arial;}") as demo:
    gr.Markdown("## 🌟 R2D2 🌟")
    gr.Markdown("Enter your review, and we'll analyze its sentiment!")

    with gr.Row():
        input_text = gr.Textbox(
            label="📝 Enter your review",
            lines=3,
            max_lines=5,
            interactive=True,
            placeholder="Type your review here..."
        )

    analyze_button = gr.Button("🔍 Analyze Sentiment")

    with gr.Row():
        output_label = gr.Textbox(label="🔽 Sentiment Result", interactive=False)

    analyze_button.click(predict_review, input_text, output_label)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7fd21291d8f7835edc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [20]:
# Load the rt dataset
rt = load_dataset("cornell-movie-review-data/rotten_tomatoes")
# Extract the train dataset from the rt
rottom_df = rt['train'].to_pandas()
rottom_df.head()

README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

text  label
0  the rock is destined to be the 21st century's ...      1
1  the gorgeously elaborate continuation of " the...      1
2                     effective but too-tepid biopic      1
3  if you sometimes like to go to the movies to h...      1
4  emerges as something rare , an issue movie tha...      1

In [22]:
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Extract reviews and labels
test_reviews = rottom_df["text"].tolist()
test_labels = rottom_df["label"].tolist()

# Tokenize and pad the reviews
sequences = tokenizer.texts_to_sequences(test_reviews)
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post")


In [23]:
# Get predictions from the model
predictions = model.predict(padded_sequences)

# Convert probabilities to binary labels (0 or 1)
predicted_labels = (predictions > 0.5).astype(int).flatten()


267/267 ━━━━━━━━━━━━━━━━━━━━ 38s 143ms/step


In [24]:
# Compute accuracy
accuracy = accuracy_score(test_labels, predicted_labels)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:\n", classification_report(test_labels, predicted_labels))

# Print confusion matrix
print("Confusion Matrix:\n", confusion_matrix(test_labels, predicted_labels))


Accuracy: 0.7262602579132473
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.69      0.72      4265
           1       0.71      0.76      0.74      4265

    accuracy                           0.73      8530
   macro avg       0.73      0.73      0.73      8530
weighted avg       0.73      0.73      0.73      8530

Confusion Matrix:
 [[2935 1330]
 [1005 3260]]


Skywalker Model

In [35]:
# Duplicate data
sky_imdb_df = data.copy()

In [36]:
# Change the name of the review column to "text" and the sentiment column to "label"
sky_imdb_df.rename(columns={'review': 'text', 'sentiment': 'label'}, inplace=True)
sky_imdb_df.head()

text  label
0  One of the other reviewers has mentioned that ...      1
1  A wonderful little production. <br /><br />The...      1
2  I thought this was a wonderful way to spend ti...      1
3  Basically there's a family where a little boy ...      0
4  Petter Mattei's "Love in the Time of Money" is...      1

In [37]:
# Set the features variable to the "text" column
X = sky_imdb_df['text']
# Set the target variable to the "label" column
y = sky_imdb_df['label']
# Split the data into train, test, split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
custom_stopwords = ['a', 'about', 'an', 'and', 'are', 'as', 'at', 'be', 'been', 'but', 'by', 'can', \
             'even', 'ever', 'for', 'from', 'get', 'had', 'has', 'have', 'he', 'her', 'hers', 'his', \
             'how', 'i', 'if', 'in', 'into', 'is', 'it', 'its', 'just', 'me', 'my', 'of', 'on', 'or', \
             'see', 'seen', 'she', 'so', 'than', 'that', 'the', 'their', 'there', 'they', 'this', \
             'to', 'was', 'we', 'were', 'what', 'when', 'which', 'who', 'will', 'with', 'you']

In [39]:
# Build a pipeline using TfidfVectorizer(), with custom_stopwords and LinearSVC
text_clf = Pipeline([('tfidf', TfidfVectorizer(stop_words=custom_stopwords, ngram_range=(1,2), max_df=0.9, min_df=2, sublinear_tf=True)),
                     ('clf', LinearSVC(C=1, loss='squared_hinge', penalty='l2', dual=True))])

# Fit the data to the model
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.9, min_df=2, ngram_range=(1, 2),
                                 stop_words=['a', 'about', 'an', 'and', 'are',
                                             'as', 'at', 'be', 'been', 'but',
                                             'by', 'can', 'even', 'ever', 'for',
                                             'from', 'get', 'had', 'has',
                                             'have', 'he', 'her', 'hers', 'his',
                                             'how', 'i', 'if', 'in', 'into',
                                             'is', ...],
                                 sublinear_tf=True)),
                ('clf', LinearSVC(C=1, dual=True))])

In [40]:
# Validate the model by checking the model accuracy with model.score
print('Train accuracy:', text_clf.score(X_train, y_train))
print('Test accuracy:', text_clf.score(X_test, y_test))

Train accuracy: 0.9998
Test accuracy: 0.9171


In [41]:
# Create a confusion matrix on the test data and predictions
predictions = text_clf.predict(X_test)
print(metrics.confusion_matrix(y_test,predictions))
print(metrics.classification_report(y_test,predictions))
print(metrics.accuracy_score(y_test,predictions))

[[4508  453]
 [ 376 4663]]
              precision    recall  f1-score   support

           0       0.92      0.91      0.92      4961
           1       0.91      0.93      0.92      5039

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000

0.9171


In [42]:
# Test a sample review "I thought the movie was slow and I wasn't very engaged"
sample = text_clf.predict(["I thought the movie was slow and I wasn't very engaged"])
sample

array([0])

In [43]:
# Test another sample review "This movie brought me so much joy. I loved the part with the animals"
sample = text_clf.predict(["This movie brought me so much joy. I loved the part with the animals"])
sample

array([1])

In [44]:
# Save the model
vectorizer = text_clf.named_steps['tfidf']
joblib.dump(vectorizer, 'vectorizer.pkl')
model = text_clf.named_steps['clf']
joblib.dump(model, 'model.pkl')

['model.pkl']

In [45]:
#Load the saved model
new_vectorizer = joblib.load('vectorizer.pkl')
new_model = joblib.load('model.pkl')

In [46]:
# Load the rt dataset
rt = load_dataset("cornell-movie-review-data/rotten_tomatoes")

In [47]:
# Extract the train dataset from the rt
rt_df = rt['train'].to_pandas()

In [48]:
rt_df.head()

text  label
0  the rock is destined to be the 21st century's ...      1
1  the gorgeously elaborate continuation of " the...      1
2                     effective but too-tepid biopic      1
3  if you sometimes like to go to the movies to h...      1
4  emerges as something rare , an issue movie tha...      1

In [49]:
# Assuming the text column is named 'review'
new_texts = rt_df['text'].astype(str)  # Convert to string if necessary

# Transform using the existing vectorizer
new_features = new_vectorizer.transform(new_texts)

In [50]:
# Get actual labels
y_true = rt_df['label']

# Predict using the trained model
y_pred = new_model.predict(new_features)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Detailed evaluation
print(classification_report(y_true, y_pred))

Accuracy: 0.7462
              precision    recall  f1-score   support

           0       0.77      0.70      0.73      4265
           1       0.72      0.80      0.76      4265

    accuracy                           0.75      8530
   macro avg       0.75      0.75      0.75      8530
weighted avg       0.75      0.75      0.75      8530



In [51]:
# Load your vectorizer and transform the dataset
vectorizer = joblib.load('vectorizer.pkl')
X = vectorizer.transform(rt_df['text'])  # Text features
y = rt_df['label']  # Target labels

# Initialize MultinomialNB
model = MultinomialNB()

# Perform 5-fold cross-validation (you can adjust the number of folds)
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

# Output the accuracy scores for each fold
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean accuracy: {cv_scores.mean()} ± {cv_scores.std()}")

Cross-validation scores: [0.76143025 0.77784291 0.7919109  0.78077374 0.77256741]
Mean accuracy: 0.7769050410316529 ± 0.009993063925498744


In [52]:
#Download Vader package
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [53]:
# Better gradio interface


# Sentiment Analysis Function returning sentiment and score
def vader_review(text):
    analyzer = SentimentIntensityAnalyzer()
    score = analyzer.polarity_scores(text)['compound']
    if score >= 0.05:
        return "😊 Positive", score
    elif score <= -0.05:
        return "😠 Negative", score
    else:
        return "😐 Neutral", score

# Improved Gradio Interface
with gr.Blocks(css=".gradio-container {background-color: #df3120; font-family: Arial;}") as demo:
    gr.Markdown("## 🌟 Vader 🌟")
    gr.Markdown("Enter your review, and we'll analyze its sentiment!")

    with gr.Row():
        input_text = gr.Textbox(
            label="📝 Enter your review",
            lines=3,
            max_lines=5,
            interactive=True,
            placeholder="Type your review here..."
        )

    analyze_button = gr.Button("🔍 Analyze Sentiment")

    with gr.Row():
        output_label = gr.Textbox(label="🔽 Sentiment Result", interactive=False)

    analyze_button.click(vader_review, input_text, output_label)

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d7cf757c35838543c0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [77]:
# # Initialize VADER
analyzer = SentimentIntensityAnalyzer()

# Function to classify sentiment as only Positive or Negative
def get_vader_sentiment(text):
    score = analyzer.polarity_scores(str(text))["compound"]
    # Return 1 for positive and 0 for negative to match y_true
    return 1 if score >= 0 else 0

# Apply classification to the "text" column, not the "label" column
sky_imdb_df["predicted_label"] = sky_imdb_df["text"].apply(get_vader_sentiment)

# Print classification report (assuming 'actual_label' exists)
print(classification_report(sky_imdb_df["label"],sky_imdb_df["predicted_label"]))

              precision    recall  f1-score   support

           0       0.79      0.53      0.64     25000
           1       0.65      0.86      0.74     25000

    accuracy                           0.70     50000
   macro avg       0.72      0.70      0.69     50000
weighted avg       0.72      0.70      0.69     50000



In [78]:
# Apply classification to the "text" column, not the "label" column
rottom_df["predicted_label"] = rottom_df["text"].apply(get_vader_sentiment)

# Print classification report (assuming 'actual_label' exists)
print(classification_report(rottom_df["label"],rottom_df["predicted_label"]))

              precision    recall  f1-score   support

           0       0.70      0.40      0.50      4265
           1       0.58      0.83      0.68      4265

    accuracy                           0.61      8530
   macro avg       0.64      0.61      0.59      8530
weighted avg       0.64      0.61      0.59      8530



In [55]:
# Sentiment Analysis Function returning sentiment and score using text_clf
def mj_review(text):
    score = text_clf.predict([text])[0]
    if score == 1:
        return "😊 Positive"
    elif score == 0:
        return "😠 Negative"

# Improved Gradio Interface
with gr.Blocks(css=".gradio-container {background-color: #3e8b4f; font-family: Arial;}") as demo:
    gr.Markdown("## 🌟 Skywalker 🌟")
    gr.Markdown("Enter your review, and we'll analyze its sentiment!")

    with gr.Row():
        input_text = gr.Textbox(
            label="📝 Enter your review",
            lines=3,
            max_lines=5,
            interactive=True,
            placeholder="Type your review here..."
        )

    analyze_button = gr.Button("🔍 Analyze Sentiment")

    with gr.Row():
        output_label = gr.Textbox(label="🔽 Sentiment Result", interactive=False)

    analyze_button.click(mj_review, input_text, output_label)

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3829bd353b5891b653.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [56]:
# Import rotten tomatoes dataset
splits = {'train': 'train.parquet', 'validation': 'validation.parquet', 'test': 'test.parquet'}
df_reviews = pd.read_parquet("hf://datasets/cornell-movie-review-data/rotten_tomatoes/" + splits["train"], engine='fastparquet')

In [57]:
# Rename 'text' column
df_reviews = df_reviews.rename(columns={'text': 'review'})

In [58]:
import spacy

# Load English NLP model
nlp = spacy.load("en_core_web_sm")

# List of movie reviews
reviews = df_reviews["review"].tolist()

In [67]:
# Process each review and extract entities

for review in reviews:
    doc = nlp(review)
    print(f"Review: {review}")
    for ent in doc.ents:
        print(f"  Entity: {ent.text}, Label: {ent.label_}")
    print("-" * 50)


Review: the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
  Entity: the 21st century's, Label: DATE
  Entity: jean, Label: NORP
--------------------------------------------------
Review: the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth .
  Entity: peter jackson, Label: PERSON
  Entity: tolkien, Label: PERSON
--------------------------------------------------
Review: effective but too-tepid biopic
--------------------------------------------------
Review: if you sometimes like to go to the movies to have fun , wasabi is a good place to start .
--------------------------------------------------
Review: emerges as something rare , an issue movie that's so honest and keenly observed that it 

KeyboardInterrupt: 

In [68]:
# Convert column to list
reviews_imdb = data["review"].tolist()
# Process each review and extract entities
for review in reviews_imdb:
    doc = nlp(review)
    print(f"Review: {review}")
    for ent in doc.ents:
        print(f"  Entity: {ent.text}, Label: {ent.label_}")
    print("-" * 50)

Review: One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to

KeyboardInterrupt: 

In [69]:
# Initialize 'count' column to None
df_reviews['count'] = None


In [70]:
# Process each review and count the number of PERSON entities
for review in reviews:
    count = 0
    doc = nlp(review)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            count += 1
    df_reviews['count'][review] = count

In [71]:
# Iterate through the DataFrame using `iterrows()`
for idx, row in df_reviews.iterrows():
    count = 0
    doc = nlp(row["review"])  # Process the review text
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            count += 1
    df_reviews.at[idx, "count"] = count  # Assign count correctly
print(df_reviews)

                                                 review  label count
0     the rock is destined to be the 21st century's ...      1     0
1     the gorgeously elaborate continuation of " the...      1     2
2                        effective but too-tepid biopic      1     0
3     if you sometimes like to go to the movies to h...      1     0
4     emerges as something rare , an issue movie tha...      1     0
...                                                 ...    ...   ...
8525  any enjoyment will be hinge from a personal th...      0     0
8526  if legendary shlockmeister ed wood had ever ma...      0     1
8527  hardly a nuanced portrait of a young woman's b...      0     0
8528    interminably bleak , to say nothing of boring .      0     0
8529  things really get weird , though not particula...      0     0

[8530 rows x 3 columns]


In [72]:
# Filter the DataFrame for positive reviews
df_reviews_positive = df_reviews.drop(df_reviews[df_reviews['label'] == 0].index)
df_reviews_positive['count'].value_counts()

count
0    3624
1     539
2      84
3      13
4       3
5       2
Name: count, dtype: int64

In [73]:
# Filter the DataFrame for negative reviews
df_reviews_negative = df_reviews.drop(df_reviews[df_reviews['label'] == 1].index)
df_reviews_negative['count'].value_counts()

count
0    3725
1     460
2      68
3      10
4       2
Name: count, dtype: int64

In [74]:
# Initialize 'all_entities_count' column to None
df_reviews['all_entities_count'] = None

In [75]:
# Process each review and count all entities
for idx, row in df_reviews.iterrows():
    count = 0
    doc = nlp(row["review"])
    for ent in doc.ents:
        count += 1
    df_reviews.at[idx, "all_entities_count"] = count
print(df_reviews)

                                                 review  label count  \
0     the rock is destined to be the 21st century's ...      1     0   
1     the gorgeously elaborate continuation of " the...      1     2   
2                        effective but too-tepid biopic      1     0   
3     if you sometimes like to go to the movies to h...      1     0   
4     emerges as something rare , an issue movie tha...      1     0   
...                                                 ...    ...   ...   
8525  any enjoyment will be hinge from a personal th...      0     0   
8526  if legendary shlockmeister ed wood had ever ma...      0     1   
8527  hardly a nuanced portrait of a young woman's b...      0     0   
8528    interminably bleak , to say nothing of boring .      0     0   
8529  things really get weird , though not particula...      0     0   

     all_entities_count  
0                     2  
1                     2  
2                     0  
3                     0  
4    

In [ ]:
# Filter the DataFrame for positive reviews
df_reviews_positive2 = df_reviews.drop(df_reviews[df_reviews['label'] == 0].index)
df_reviews_positive2['all_entities_count'].value_counts()

all_entities_count
0    2639
1    1108
2     381
3     104
4      22
5       8
6       2
7       1
Name: count, dtype: int64

In [ ]:
# Filter the DataFrame for negative reviews
df_reviews_negative2 = df_reviews.drop(df_reviews[df_reviews['label'] == 1].index)
df_reviews_negative2['all_entities_count'].value_counts()

all_entities_count
0    2850
1    1017
2     286
3      83
4      24
5       4
6       1
Name: count, dtype: int64